<h1> Import Lib </h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import skew
from scipy import stats
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_regression, f_regression
from sklearn import ensemble
from sklearn.metrics import mean_squared_error


<h1> Loda Data </h1>

In [3]:
df = pd.read_csv('/kaggle/input/house-prices-data/train.csv')
dt = pd.read_csv('/kaggle/input/house-prices-data/test.csv')

In [4]:
df.shape,dt.shape

((1460, 81), (1459, 80))

<h1>Check Null</h1>

In [5]:
(((df.isnull().sum())*100)/len(df)).sort_values(
            ascending = False, kind = 'mergesort').head(15)

PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
FireplaceQu     47.260274
LotFrontage     17.739726
GarageCond       5.547945
GarageQual       5.547945
GarageFinish     5.547945
GarageYrBlt      5.547945
GarageType       5.547945
BsmtFinType2     2.602740
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtCond         2.534247
dtype: float64

In [6]:
(((dt.isnull().sum())*100)/len(dt)).sort_values(
            ascending = False, kind = 'mergesort').head(30)

PoolQC          99.794380
MiscFeature     96.504455
Alley           92.666210
Fence           80.123372
FireplaceQu     50.034270
LotFrontage     15.558602
GarageCond       5.346127
GarageQual       5.346127
GarageFinish     5.346127
GarageYrBlt      5.346127
GarageType       5.209047
BsmtCond         3.084304
BsmtExposure     3.015764
BsmtQual         3.015764
BsmtFinType2     2.878684
BsmtFinType1     2.878684
MasVnrType       1.096642
MasVnrArea       1.028101
MSZoning         0.274160
Functional       0.137080
BsmtHalfBath     0.137080
BsmtFullBath     0.137080
Utilities        0.137080
SaleType         0.068540
GarageArea       0.068540
GarageCars       0.068540
KitchenQual      0.068540
TotalBsmtSF      0.068540
BsmtUnfSF        0.068540
BsmtFinSF2       0.068540
dtype: float64

In [7]:
df.drop(['Id','PoolQC','MiscFeature','Alley','Fence'],axis=1,inplace=True)
dt.drop(['Id','PoolQC','MiscFeature','Alley','Fence'],axis=1,inplace=True)

In [8]:
y = df['SalePrice']
df.drop(['SalePrice'], axis = 1, inplace = True)
data = pd.concat([df,dt], axis = 0)
data.shape

(2919, 75)

In [9]:
data.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,2918.000000,...,2918.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000
mean,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,49.582248,...,472.874572,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737
std,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,169.205611,...,215.394815,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,0.000000,...,320.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,0.000000,...,480.000000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,0.000000,...,576.000000,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1526.000000,...,1488.000000,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


<h1> Replace Null </h1>

In [10]:
year_all = ['YearBuilt', 'YearRemodAdd','YrSold','MoSold','GarageYrBlt']
for i in data:
  if (data[i].dtypes == object and i != 'FireplaceQu') or i in year_all:
    data[i] = data[i].fillna(data[i].mode()[0])

In [11]:
data['BsmtHalfBath'] = data['BsmtHalfBath'].fillna(data['BsmtHalfBath'].mode()[0])
data['BsmtFullBath'] = data['BsmtFullBath'].fillna(data['BsmtFullBath'].mode()[0])
data['GarageCars'] = data['GarageCars'].fillna(data['GarageCars'].mode()[0])

In [12]:
for i in data:
  if data[i].dtypes != object:
     data[i] = data[i].fillna(data[i].mean())
     data[i] = data[i].astype('float64')

In [13]:
# Replace Null using Classification

classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.01, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(ccp_alpha=0.0,max_depth=20, n_estimators=4500),
    GaussianNB(),
    GradientBoostingClassifier(learning_rate=0.03),
    AdaBoostClassifier(learning_rate=0.6)]
#Using only numerical features
fireplace_data = data.select_dtypes(exclude=['object'])
fireplace_data['FireplaceQu'] = data['FireplaceQu'].copy()

def classify_missing(all_int_dataTe):
  fire_train = pd.DataFrame()
  fire_test = pd.DataFrame()
  null_row_list = list(all_int_dataTe[all_int_dataTe['FireplaceQu'].isnull()].index.tolist())
  col = list(all_int_dataTe.columns)
  k = -1
  for j in null_row_list:
    try:
      k+=1 
      null_row_value = all_int_dataTe.iloc[j]
      null_row_value = list(null_row_value.values)
      for i in range(len(null_row_value)):
        fire_test.loc[k,col[i]] = null_row_value[i]
    except:
      continue
  fire_train = all_int_dataTe.dropna()
  fire_test = fire_test.drop(['FireplaceQu'],axis = 1)

  f_train = fire_train.drop(['FireplaceQu'],axis = 1)
  y_train = fire_train['FireplaceQu']

  
  le = preprocessing.LabelEncoder()
  le.fit(y_train)
  y_train = le.transform(y_train)

  
  X_train, X_test, y_train, y_test = train_test_split(f_train, y_train, test_size=0.1, random_state=1)
  r = 0
  for clf in classifiers:
    clf.fit(X_train,y_train)
    print(clf.__class__.__name__,' ', round(clf.score(X_test, y_test) * 100, 2))
    if r < clf.score(X_test, y_test):
      r = clf.score(X_test, y_test)
      model = clf
  print(model)
  model.fit(X_train,y_train)
  Y_prediction = model.predict(fire_test)
  res = le.inverse_transform(Y_prediction)

  for i in range(len(null_row_list)):
    try:
      all_int_dataTe.loc[null_row_list[i],'FireplaceQu'] = res[i]
    except:
      print('error')
  return all_int_dataTe['FireplaceQu']
data['FireplaceQu'] = classify_missing(fireplace_data)

LogisticRegression   61.33
KNeighborsClassifier   56.67
SVC   50.67
DecisionTreeClassifier   66.67
RandomForestClassifier   72.67
GaussianNB   10.0
GradientBoostingClassifier   70.67
AdaBoostClassifier   46.0
RandomForestClassifier(max_depth=20, n_estimators=4500)


In [14]:
(((data.isnull().sum())*100)/len(data)).sort_values(
            ascending = False, kind = 'mergesort').head(15)

SaleCondition    0.0
SaleType         0.0
YrSold           0.0
MoSold           0.0
MiscVal          0.0
PoolArea         0.0
ScreenPorch      0.0
3SsnPorch        0.0
EnclosedPorch    0.0
OpenPorchSF      0.0
WoodDeckSF       0.0
PavedDrive       0.0
GarageCond       0.0
GarageQual       0.0
GarageArea       0.0
dtype: float64